**Loading dataset**

In [61]:
##Loading Dataset
import pandas as pd
df= pd.read_csv("TRADES_CopyTr_90D_ROI.csv")

In [63]:
df.head()

,Port_IDs,Trade_History
0,3925368433214965504,NaN
1,4002413037164645377,NaN
2,3923766029921022977,NaN
3,3994879592543698688,NaN
4,3926423286576838657,NaN


In [65]:
df

,Port_IDs,Trade_History
0,3925368433214965504,NaN
1,4002413037164645377,NaN
2,3923766029921022977,NaN
3,3994879592543698688,NaN
4,3926423286576838657,NaN
...,...,...
145,4000222729738650369,NaN
146,3998659472131949824,"[{'time': 1718979385000, 'symbol': 'BTCUSDT', ..."
147,4028701921959171840,NaN
148,4014818740371615232,NaN


In [67]:
#Displays the number of null and non-null entries
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Port_IDs       150 non-null    int64 
 1   Trade_History  37 non-null     object
dtypes: int64(1), object(1)
memory usage: 2.5+ KB


**Parse the Trade_history column**

In [69]:
w_na=df.dropna(subset=["Trade_History"])

In [71]:
# Display some non-null Trade_History entries to examine the data format
df["Trade_History"].dropna().head(5).tolist()

["[{'time': 1718641182000, 'symbol': 'DOGEUSDT', 'side': 'BUY', 'price': 0.1267, 'fee': -2.43851888, 'feeAsset': 'USDT', 'quantity': 12192.5944, 'quantityAsset': 'USDT', 'realizedProfit': 803.42456999, 'realizedProfitAsset': 'USDT', 'baseAsset': 'DOGE', 'qty': 96232.0, 'positionSide': 'SHORT', 'activeBuy': False}, {'time': 1718381884000, 'symbol': 'DOGEUSDT', 'side': 'SELL', 'price': 0.13504, 'fee': -0.08412992, 'feeAsset': 'USDT', 'quantity': 168.25984, 'quantityAsset': 'USDT', 'realizedProfit': 0.0, 'realizedProfitAsset': 'USDT', 'baseAsset': 'DOGE', 'qty': 1246.0, 'positionSide': 'SHORT', 'activeBuy': False}, {'time': 1718381884000, 'symbol': 'DOGEUSDT', 'side': 'SELL', 'price': 0.13504, 'fee': -0.4939088, 'feeAsset': 'USDT', 'quantity': 987.8176, 'quantityAsset': 'USDT', 'realizedProfit': 0.0, 'realizedProfitAsset': 'USDT', 'baseAsset': 'DOGE', 'qty': 7315.0, 'positionSide': 'SHORT', 'activeBuy': False}, {'time': 1718381884000, 'symbol': 'DOGEUSDT', 'side': 'SELL', 'price': 0.13504

In [73]:
w_dup=w_na.drop_duplicates()

**Parsing into structure dataframe**

In [77]:
import ast

# Function to safely parse Trade_History
def parse_trade_history(trade_history):
    try:
        return ast.literal_eval(trade_history) if isinstance(trade_history, str) else None
    except (SyntaxError, ValueError):
        return None

# Apply parsing to the Trade_History column
df["Parsed_Trade_History"] = df["Trade_History"].apply(parse_trade_history)

# Display a sample of parsed trade history
df["Parsed_Trade_History"].dropna().head(3).tolist()


[[{'time': 1718641182000,
   'symbol': 'DOGEUSDT',
   'side': 'BUY',
   'price': 0.1267,
   'fee': -2.43851888,
   'feeAsset': 'USDT',
   'quantity': 12192.5944,
   'quantityAsset': 'USDT',
   'realizedProfit': 803.42456999,
   'realizedProfitAsset': 'USDT',
   'baseAsset': 'DOGE',
   'qty': 96232.0,
   'positionSide': 'SHORT',
   'activeBuy': False},
  {'time': 1718381884000,
   'symbol': 'DOGEUSDT',
   'side': 'SELL',
   'price': 0.13504,
   'fee': -0.08412992,
   'feeAsset': 'USDT',
   'quantity': 168.25984,
   'quantityAsset': 'USDT',
   'realizedProfit': 0.0,
   'realizedProfitAsset': 'USDT',
   'baseAsset': 'DOGE',
   'qty': 1246.0,
   'positionSide': 'SHORT',
   'activeBuy': False},
  {'time': 1718381884000,
   'symbol': 'DOGEUSDT',
   'side': 'SELL',
   'price': 0.13504,
   'fee': -0.4939088,
   'feeAsset': 'USDT',
   'quantity': 987.8176,
   'quantityAsset': 'USDT',
   'realizedProfit': 0.0,
   'realizedProfitAsset': 'USDT',
   'baseAsset': 'DOGE',
   'qty': 7315.0,
   'positi

**ROI and PnL for each account.**

In [79]:
# Flatten trade history into a structured DataFrame
trade_data = []

for _, row in df.dropna(subset=["Parsed_Trade_History"]).iterrows():
    port_id = row["Port_IDs"]
    trades = row["Parsed_Trade_History"]
    
    for trade in trades:
        trade["Port_ID"] = port_id  # Associate trade with account
        trade_data.append(trade)

# Create DataFrame from structured trade data
trade_df = pd.DataFrame(trade_data)

# Display basic info and first few rows
trade_df.info(), trade_df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3096 entries, 0 to 3095
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   time                 3096 non-null   int64  
 1   symbol               3096 non-null   object 
 2   side                 3096 non-null   object 
 3   price                3096 non-null   float64
 4   fee                  3096 non-null   float64
 5   feeAsset             3096 non-null   object 
 6   quantity             3096 non-null   float64
 7   quantityAsset        3096 non-null   object 
 8   realizedProfit       3096 non-null   float64
 9   realizedProfitAsset  3096 non-null   object 
 10  baseAsset            3096 non-null   object 
 11  qty                  3096 non-null   float64
 12  positionSide         3096 non-null   object 
 13  activeBuy            3096 non-null   bool   
 14  Port_ID              3096 non-null   int64  
dtypes: bool(1), float64(5), int64(2), obje

(None,
             time    symbol  side    price       fee feeAsset     quantity  \
 0  1718641182000  DOGEUSDT   BUY  0.12670 -2.438519     USDT  12192.59440   
 1  1718381884000  DOGEUSDT  SELL  0.13504 -0.084130     USDT    168.25984   
 2  1718381884000  DOGEUSDT  SELL  0.13504 -0.493909     USDT    987.81760   
 3  1718381884000  DOGEUSDT  SELL  0.13504 -0.337600     USDT    675.20000   
 4  1718381884000  DOGEUSDT  SELL  0.13504 -1.145004     USDT   2290.00832   
 
   quantityAsset  realizedProfit realizedProfitAsset baseAsset      qty  \
 0          USDT       803.42457                USDT      DOGE  96232.0   
 1          USDT         0.00000                USDT      DOGE   1246.0   
 2          USDT         0.00000                USDT      DOGE   7315.0   
 3          USDT         0.00000                USDT      DOGE   5000.0   
 4          USDT         0.00000                USDT      DOGE  16958.0   
 
   positionSide  activeBuy              Port_ID  
 0        SHORT      

**Sharpe Ratio and Maximum Drawdown (MDD)**

In [81]:
# Group by Port_ID to calculate financial metrics
account_metrics = trade_df.groupby("Port_ID").agg(
    Total_Investment=("quantity", "sum"),  # Total capital used in trades
    Total_Realized_Profit=("realizedProfit", "sum"),  # Net profit or loss
    Win_Trades=("realizedProfit", lambda x: (x > 0).sum()),  # Count of profitable trades
    Total_Trades=("realizedProfit", "count")  # Total trades executed
).reset_index()

# Calculate additional metrics
account_metrics["ROI"] = (account_metrics["Total_Realized_Profit"] / account_metrics["Total_Investment"]) * 100
account_metrics["Win_Rate"] = (account_metrics["Win_Trades"] / account_metrics["Total_Trades"]) * 100

# Display top accounts based on ROI
account_metrics.sort_values(by="ROI", ascending=False).head(10)


,Port_ID,Total_Investment,Total_Realized_Profit,Win_Trades,Total_Trades,ROI,Win_Rate
3,3826087012661391104,4373.742334,532.656974,63,108,12.178517,58.333333
0,3768170840939476993,2775.560153,243.668899,6,14,8.779089,42.857143
11,3977234346014419201,59166.712085,2427.292831,33,83,4.102464,39.759036
21,4029507714735307777,44938.914960,1537.899960,2,53,3.422201,3.773585
17,4023697881318718465,95292.775420,3151.950620,6,70,3.307649,8.571429
15,4004713168329653760,51922.300610,1667.198610,2,27,3.210949,7.407407
25,4030708912786799105,50385.288770,1612.843770,2,60,3.201021,3.333333
16,4023697433751327232,75586.884800,2408.224000,4,42,3.186034,9.523810
33,4039129759104249600,39977.784200,1264.289200,59,133,3.162479,44.360902
22,4030392204221219329,43976.054010,1347.758010,8,85,3.064754,9.411765


**Calculating MDD, Daily Return and Ranking top 20 accounts on the basis of Sharpe Ratio**

In [83]:
# Convert time to datetime format
trade_df["time"] = pd.to_datetime(trade_df["time"], unit="ms")

# Compute daily returns per account
trade_df["daily_return"] = trade_df.groupby(["Port_ID", trade_df["time"].dt.date])["realizedProfit"].transform("sum")

# Calculate Sharpe Ratio and Maximum Drawdown (MDD) per account
sharpe_mdd_metrics = trade_df.groupby("Port_ID").agg(
    Mean_Daily_Return=("daily_return", "mean"),
    Std_Daily_Return=("daily_return", "std"),
    Max_Drawdown=("daily_return", lambda x: (x.cumsum().cummax() - x.cumsum()).max())  # Peak-to-trough loss
).reset_index()

# Calculate Sharpe Ratio (assume risk-free rate = 0)
sharpe_mdd_metrics["Sharpe_Ratio"] = sharpe_mdd_metrics["Mean_Daily_Return"] / sharpe_mdd_metrics["Std_Daily_Return"]

# Merge with account metrics
account_metrics = account_metrics.merge(sharpe_mdd_metrics, on="Port_ID")

# Display updated metrics
account_metrics.sort_values(by="Sharpe_Ratio", ascending=False).head(10)


,Port_ID,Total_Investment,Total_Realized_Profit,Win_Trades,Total_Trades,ROI,Win_Rate,Mean_Daily_Return,Std_Daily_Return,Max_Drawdown,Sharpe_Ratio
28,4033639786957934336,402568.45661,2848.141857,25,149,0.707493,16.778523,602.593336,181.119225,0.0,3.327053
17,4023697881318718465,95292.77542,3151.950620,6,70,3.307649,8.571429,718.881079,224.778844,0.0,3.198171
29,4035430878731345664,130569.53840,2493.750420,43,89,1.909902,48.314607,1373.280407,436.644048,0.0,3.145080
15,4004713168329653760,51922.30061,1667.198610,2,27,3.210949,7.407407,762.064857,274.742862,0.0,2.773739
36,4041860229502600193,173806.66200,2327.336000,38,97,1.339037,39.175258,1123.545907,432.798341,0.0,2.596003
35,4040843843196854529,218331.08006,2151.704060,19,59,0.985523,32.203390,1229.732245,569.154510,0.0,2.160630
19,4029422834086627072,215737.03283,2799.902550,38,102,1.297831,37.254902,713.857082,351.753280,0.0,2.029426
30,4036548677626662656,586961.16148,3296.876680,68,147,0.561686,46.258503,1495.709471,740.343314,0.0,2.020292
20,4029506971304830209,79154.03182,2413.654000,3,52,3.049313,5.769231,619.586498,342.674895,0.0,1.808088
33,4039129759104249600,39977.78420,1264.289200,59,133,3.162479,44.360902,480.101426,271.260310,0.0,1.769892


**Final ranking with weighted Scoring**

In [85]:
from sklearn.preprocessing import MinMaxScaler

# Define weights for the ranking system
weights = {
    "ROI": 0.4,
    "Sharpe_Ratio": 0.3,
    "Win_Rate": 0.2,
    "Total_Realized_Profit": 0.1
}

# Normalize metrics using MinMaxScaler (0 to 1 scale)
scaler = MinMaxScaler()
normalized_metrics = account_metrics.copy()
normalized_metrics[["ROI", "Sharpe_Ratio", "Win_Rate", "Total_Realized_Profit"]] = scaler.fit_transform(
    normalized_metrics[["ROI", "Sharpe_Ratio", "Win_Rate", "Total_Realized_Profit"]]
)

# Compute final weighted score
normalized_metrics["Score"] = (
    weights["ROI"] * normalized_metrics["ROI"] +
    weights["Sharpe_Ratio"] * normalized_metrics["Sharpe_Ratio"] +
    weights["Win_Rate"] * normalized_metrics["Win_Rate"] +
    weights["Total_Realized_Profit"] * normalized_metrics["Total_Realized_Profit"]
)

# Rank accounts based on the final score
normalized_metrics = normalized_metrics.sort_values(by="Score", ascending=False).reset_index(drop=True)

# Select top 20 accounts
top_20_accounts = normalized_metrics.head(20)

# Display top ranked accounts
top_20_accounts[["Port_ID", "ROI", "Sharpe_Ratio", "Win_Rate", "Total_Realized_Profit", "Score"]]


,Port_ID,ROI,Sharpe_Ratio,Win_Rate,Total_Realized_Profit,Score
0,3826087012661391104,1.000000,0.259504,0.816667,0.065340,0.647719
1,4035430878731345664,0.164309,0.943098,0.676404,0.245947,0.508529
2,3768170840939476993,0.723344,0.260972,0.600000,0.038726,0.491502
3,4023697881318718465,0.278062,0.959699,0.120000,0.306564,0.453791
4,4041860229502600193,0.117850,0.771406,0.548454,0.230621,0.411315
5,4033639786957934336,0.066453,1.000000,0.234899,0.278585,0.401420
6,4039129759104249600,0.266248,0.513087,0.621053,0.132720,0.397907
7,4004713168329653760,0.270192,0.826982,0.103704,0.169826,0.393895
8,3956048468100538880,0.099582,0.370874,1.000000,0.142783,0.365373
9,4036548677626662656,0.054587,0.591385,0.647619,0.319911,0.360765
